In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import re
from scipy import optimize
import fastcluster
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import pickle as pkl
from matplotlib.lines import Line2D
import collections
from collections import defaultdict
from adjustText import adjust_text
from sklearn.manifold import TSNE
from scipy import optimize


In [3]:
os.getcwd()
os.chdir('/home/jiaozhenna/methylation/project/EM_seq/analysis/deconv_analysis/TAPS_data/data/')
sample_df=pd.read_csv('./sample_mixture_cpg_df_keepna.csv',sep=',',index_col=None)
alter_df=pd.read_csv('/home/jiaozhenna/methylation/project/EM_seq/analysis/deconv_analysis/TAPS_data/ref_altar/cfTAPS_Enhancer_Tissue_Atlas_Final.csv',sep='\t',index_col=None)

sample_df=sample_df.set_index('chr_region')
alter_df=alter_df.set_index('chr_start_end')
## index is the same
common_index = sample_df.index.intersection(alter_df.index)
sample_df = sample_df.loc[common_index]
alter_df = alter_df.loc[common_index]

# sort index
sample_df = sample_df.sort_index()
alter_df = alter_df.sort_index()

print(sample_df)
print(alter_df)

                           gp01_L14-1550  gp03_L15-31  gp04_L15-12  \
chr10_100353201_100355200          0.959        0.860        0.919   
chr10_100391201_100391400          1.000        0.667        1.000   
chr10_100391201_100391400          1.000        0.667        1.000   
chr10_100391201_100391400          1.000        0.667        1.000   
chr10_100391201_100391400          1.000        0.667        1.000   
...                                  ...          ...          ...   
chr9_99814401_99815400               NaN        1.000        1.000   
chr9_99816601_99817000             0.833        0.971        1.000   
chr9_99862801_99863400               NaN        0.889        0.833   
chr9_99862801_99863400               NaN        0.889        0.833   
chr9_99966001_99968800             0.900        0.941        1.000   

                           gp05_L15-1550  gp06_L15-13  gp07_L19-5  gp08_L19-6  \
chr10_100353201_100355200          0.877        0.892       0.939       0.829 

In [4]:

#-------------------NNLS Deconvolution-------------------
sample_mixtures=pd.DataFrame([])
residuals=[]
for sample_name in sample_df:
    missing_idx=sample_df[sample_name].isnull()
    cleaned_sample_df=sample_df.loc[~missing_idx,sample_name]
    cleaned_altar_df=alter_df.loc[~missing_idx,:]

    mixture, residual = optimize.nnls(cleaned_altar_df, cleaned_sample_df)
    mixture /= np.sum(mixture)
    residuals.append(residual)
    sample_mixtures[sample_name] = mixture 
    
    sample_mixtures['atlas_tissues'] = alter_df.columns
    sample_mixtures = sample_mixtures.set_index('atlas_tissues')
    mean_residual = np.array(residuals).mean()
    # normalised_residual = np.array(residuals).mean()/merged_tissues_ratio.shape[0]
# -----write the results--------
# sample_mixtures.to_csv('./sample_mixtures.csv',sep='\t',index=True)


In [2]:
# ----------------------Do pie plot for the sample mixtures----------------------

os.getcwd()
os.chdir('/home/jiaozhenna/methylation/project/EM_seq/analysis/deconv_analysis/TAPS_data/data/')
select_GP=pd.read_csv('./pie_dat_GP.csv',sep='\t',index_col=0)
select_GC=pd.read_csv('./pie_dat_GC.csv',sep='\t',index_col=0)


